In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
import string


In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['label']=0
true['label']=1

In [6]:
news = pd.concat([fake,true],axis = 0)

In [7]:
news.shape

(44898, 5)

In [8]:
news.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [9]:
news.sample(10)

,title,text,subject,date,label
4665,Wells Fargo To Give Strikingly Fat Reward To ...,"Wells Fargo, one of the biggest banks in the U...",News,"September 12, 2016",0
8295,Trump's immigration pitch falls flat with Repu...,(Reuters) - Republican presidential candidate ...,politicsNews,"September 1, 2016",1
8579,Clinton leads Trump by 6 points after Democrat...,NEW YORK (Reuters) - Democratic presidential ...,politicsNews,"July 29, 2016",1
9269,JOHN BOLTON GETS IT: ‘This is the First Attemp...,Ambassador Bolton gets it! He and so many othe...,politics,"Dec 5, 2017",0
21093,"BRITISH MAN GOES UNDERCOVER, Makes Chilling Vi...","If she doesn t wear a hijab, we hit her This ...",left-news,"Jan 20, 2016",0
3793,Trump reassures farmers immigration crackdown ...,WASHINGTON/SAN FRANCISCO (Reuters) - President...,politicsNews,"May 15, 2017",1
19462,Trump says will be putting more sanctions on N...,NEW YORK (Reuters) - President Donald Trump sa...,worldnews,"September 21, 2017",1
13147,Leftist leader backs Guillier in Chile's run-o...,SANTIAGO (Reuters) - The leader of an influent...,worldnews,"December 5, 2017",1
16750,Toxic fumes keep EU summit venue shut for anot...,BRUSSELS (Reuters) - The building that houses ...,worldnews,"October 23, 2017",1
17034,BREAKING: IRAN REJECTED U.S. NUKE DEAL As They...,Say what?The Iranian parliament has rejected t...,Government News,"Oct 15, 2015",0


In [10]:
news['content'] = news['title']+" "+news['text']

In [11]:
news

,title,text,subject,date,label,content
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1,'Fully committed' NATO backs new U.S. approach...
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1,LexisNexis withdrew two products from Chinese ...
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1,Minsk cultural hub becomes haven from authorit...
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1,Vatican upbeat on possibility of Pope Francis ...


In [12]:
news['content']

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorit...
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: content, Length: 44898, dtype: object

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def stemming(content):
    if not isinstance(content, str):
        return ""
    # Remove non-alphabetic characters
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if word not in stop_words]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news['content'] = news['content'].apply(stemming)

In [15]:
# Check for missing columns and missing values 
required_columns = ['title', 'text']
missing_columns = [col for col in required_columns if col not in news.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    print("All required columns are present.")
    print("Missing values in 'title':", news['title'].isna().sum())
    print("Missing values in 'text':", news['text'].isna().sum())

All required columns are present.
Missing values in 'title': 0
Missing values in 'text': 0


In [16]:
news['content']

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3        trump obsess even obama name code websit imag ...
4        pope franci call donald trump christma speech ...
                               ...                        
21412    fulli commit nato back new u approach afghanis...
21413    lexisnexi withdrew two product chines market l...
21414    minsk cultur hub becom author minsk reuter sha...
21415    vatican upbeat possibl pope franci visit russi...
21416    indonesia buy billion worth russian jet jakart...
Name: content, Length: 44898, dtype: object

In [17]:
x = news['content'].values
y = news['label'].values

In [18]:
print(x)

['donald trump send embarrass new year eve messag disturb donald trump wish american happi new year leav instead give shout enemi hater dishonest fake news media former realiti show star one job countri rapidli grow stronger smarter want wish friend support enemi hater even dishonest fake news media happi healthi new year presid angri pant tweet great year america countri rapidli grow stronger smarter want wish friend support enemi hater even dishonest fake news media happi healthi new year great year america donald j trump realdonaldtrump decemb trump tweet went welll expect kind presid send new year greet like despic petti infantil gibberish trump lack decenc even allow rise gutter long enough wish american citizen happi new year bishop talbert swan talbertswan decemb one like calvin calvinstowel decemb impeach would make great year america also accept regain control congress miranda yaver mirandayav decemb hear talk includ mani peopl hate wonder hate alan sandov alansandov decemb us

In [19]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [20]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6904081 stored elements and shape (44898, 89868)>
  Coords	Values
  (0, 473)	0.029901721489513068
  (0, 1739)	0.05138826757528657
  (0, 1749)	0.08889530050540473
  (0, 2170)	0.02325325572885059
  (0, 2301)	0.015138822832065134
  (0, 2416)	0.030776265805381363
  (0, 2562)	0.0649607608565772
  (0, 2575)	0.036268116701632415
  (0, 2917)	0.03962726610765806
  (0, 3031)	0.037214846319311164
  (0, 3148)	0.04785142759816799
  (0, 3446)	0.0378234764133366
  (0, 5450)	0.04443233203028609
  (0, 6676)	0.023831801439703443
  (0, 7870)	0.05339370423539277
  (0, 9454)	0.028369643121826767
  (0, 10508)	0.06187015374653721
  (0, 11108)	0.06851913882817916
  (0, 11109)	0.07878871172088633
  (0, 12611)	0.04579308567051352
  (0, 13752)	0.02855422985589162
  (0, 13991)	0.03126141595114052
  (0, 14709)	0.02556349746054079
  (0, 15149)	0.024568186836617927
  (0, 15392)	0.024752264230251966
  :	:
  (44897, 72790)	0.14107018657201614
  (44897, 7348

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, stratify=y, random_state=1)

In [22]:
x_train.shape

(35918, 89868)

In [23]:
x_test.shape

(8980, 89868)

In [24]:
lr_model = LogisticRegression()
lr_model.fit(x_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [25]:
train_y_pred = lr_model.predict(x_train)
print("Train accuracy: ",accuracy_score(train_y_pred,y_train))

Train accuracy:  0.9915084358817307


In [26]:
test_y_pred = lr_model.predict(x_test)
print("Train accuracy: ",accuracy_score(test_y_pred,y_test))

Train accuracy:  0.9875278396436525


In [27]:
#Prediction system

input_data = x_test[570]
prediction = lr_model.predict(input_data)
if prediction[0]==1:
    print('Real News')
else:
    print('Fake News')


Fake News


In [28]:
news['content']

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3        trump obsess even obama name code websit imag ...
4        pope franci call donald trump christma speech ...
                               ...                        
21412    fulli commit nato back new u approach afghanis...
21413    lexisnexi withdrew two product chines market l...
21414    minsk cultur hub becom author minsk reuter sha...
21415    vatican upbeat possibl pope franci visit russi...
21416    indonesia buy billion worth russian jet jakart...
Name: content, Length: 44898, dtype: object

In [29]:
news.sample(10)

,title,text,subject,date,label,content
1564,U.S. House Republican speaker Ryan opposes pro...,WASHINGTON (Reuters) - U.S. House Republican S...,politicsNews,"September 26, 2017",1,u hous republican speaker ryan oppos protest a...
9840,Top House Democrat: Hopeful action on Puerto R...,WASHINGTON (Reuters) - House of Representative...,politicsNews,"April 21, 2016",1,top hous democrat hope action puerto rico bill...
19293,"VIETNAM VET, MILITARY “Lifer” DESTROYS Crybaby...",A powerful letter from a lifer in the milita...,left-news,"Jan 17, 2017",0,vietnam vet militari lifer destroy crybabi dem...
7682,Why The Secret Service Is Investigating Glenn...,The United States Secret Service is investigat...,News,"March 5, 2016",0,secret servic investig glenn beck video unit s...
16962,BREAKING: Major Health Insurance Carrier May P...,It turns out the words affordable and care ...,Government News,"Nov 19, 2015",0,break major health insur carrier may pull obam...
10290,WATCH: SARAH HUCKABEE-SANDERS Mocks Media For ...,Sarah Huckabee Sanders has taken control of th...,politics,"Jul 28, 2017",0,watch sarah huckabe sander mock media ignor pu...
10445,Trump presidency would not hurt 2024 Olympics ...,"BEVERLY HILLS, Calif (Reuters) - Even as U.S. ...",politicsNews,"March 9, 2016",1,trump presid would hurt olymp bid la mayor bev...
6295,Trump Campaign Accidentally Reveals How Donal...,Oops. While the media is still talking about t...,News,"May 18, 2016",0,trump campaign accident reveal donald trump ev...
5838,Trump Epically Owned Himself In A Tweet And T...,Donald Trump tweets out a number of terrible o...,News,"June 17, 2016",0,trump epic own tweet internet fun donald trump...
7155,Senior U.S. officials recommend removal of NSA...,WASHINGTON (Reuters) - The heads of the Pentag...,politicsNews,"November 19, 2016",1,senior u offici recommend remov nsa director s...


In [30]:
joblib.dump(vectorizer,"vectorizer_new.jb")
joblib.dump(lr_model,"lr_model.jb")

['lr_model.jb']

In [31]:
import sklearn
print(sklearn.__version__)

1.7.1
